In [1]:
import os, sys, subprocess
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
""" Function to convert sector to polar angle """
def sector2polar(sector):
    polarAngle = 270.0 - sector # Returns in the range [-90, +270] degrees rather than [-180, +180]
    if polarAngle >=180.0 and polarAngle <= 270.0:
        polarAngle = polarAngle - 360.0
    return polarAngle

""" Compute axial velocity """
def axialVel(U, V, sector):
    polarAngle = sector2polar(sector)
    fDir = np.deg2rad(polarAngle)
    #print 'polar angle: ', polarAngle, 'deg = ', fDir, ' radians'
    Uax = U*np.cos(fDir) + V*np.sin(fDir)
    #print U, V, Uax
    return Uax

In [3]:
def readU(test_path, fileName):
    probesU = pd.read_csv(os.path.join(test_path, fileName), sep='\s+', skiprows=0,
                                 names=['x', 'y', 'z', 'U', 'V', 'W'])
    return probesU

def readUprime(test_path, fileName):
    probesUp = pd.read_csv(os.path.join(test_path, fileName), sep='\s+', skiprows=0,
                                 names=['x', 'y', 'z', 'Uxx', 'Uyy', 'Uzz', 'Uxy', 'Uyz', 'Uxz'])
    return probesUp

def readkOmega(test_path, fileName):
    probesk = pd.read_csv(os.path.join(test_path, fileName), sep='\s+', skiprows=0,
                                 names=['x', 'y', 'z', 'k', 'p', 'omega', 'nuSgs'])
    return probesk

def readkEps(test_path, fileName):
    probesk = pd.read_csv(os.path.join(test_path, fileName), sep='\s+', skiprows=0,
                                 names=['x', 'y', 'z', 'k', 'p', 'nuSgs'])
    return probesk

def readkEpsRANS(test_path, fileName):
    probesk = pd.read_csv(os.path.join(test_path, fileName), sep='\s+', skiprows=0,
                                 names=['x', 'y', 'z', 'k', 'p'])
    return probesk

In [4]:
def plotSimData(probes,caseInd,probeIndex,lineDef,xVar,yVar,k_res = False):
    for caseI in caseInd:
        #Extract data for the case and the specific probes
        probeInd = probeIndex[caseI][lineDef]
        #print '... ... yVar: ',yVar, 'case name: ', caseName[caseI], ', probeInd: ', probeInd
        data = probes[caseI].iloc[probeInd] if len(probeInd)>0 else probes[caseI]
        if yVar == 'k' and k_res == True:
            dataUp = probesUp[caseI].iloc[probeInd] if len(probeInd)>0 else probesUp[caseI]
            resolved_k = 0.5*(dataUp['Uxx'] + dataUp['Uyy'] + dataUp['Uzz'])
            #print 'Resolved k: ', resolved_k
            plt.plot(data[xVar], data[yVar] + resolved_k, plotStyle[caseI], label=labelText[caseI])
        else:
            plt.plot(data[xVar], data[yVar], plotStyle[caseI], label=labelText[caseI])         

In [5]:
def plotExpt(xExpt, yExpt, probeInd):
    #print 'xExpt:\n', xExpt
    #print 'yExpt:\n', yExpt
    if len(probeInd)>0:
        plt.plot(xExpt[probeInd], yExpt[probeInd], 's', color='k', label='Measured')
    else:
        plt.plot(xExpt, yExpt, 's', color='k', label='Measured')
            

In [6]:
def plotLabelEtc(xlabelText,ylabelText,plotSubDir,figName):
    plt.xlabel(xlabelText)
    plt.ylabel(ylabelText)
    #plt.legend(loc='best')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.18),
               ncol=3, fancybox=True, shadow=True)
    plotDir = os.path.join('plots', plotSubDir)
    if not os.path.exists(plotDir):
        os.makedirs(plotDir)
    plt.savefig(os.path.join(plotDir,figName))
    plt.close()

In [7]:
def plotVar(probes,caseInd,probeInd,lineDef,xVar,yVar, xlabelText,ylabelText,plotSubDir,figName,k_res = False):
    plotSimData(probes,caseInd,probeInd,lineDef,xVar,yVar,k_res = False)
    plotLabelEtc(xlabelText,ylabelText,plotSubDir,figName)

In [8]:
def plotVarExpt(probes,caseInd,probeInd,lineDef,xVar,yVar, xlabelText,ylabelText,plotSubDir,figName, xExpt,yExpt,k_res = False):
    plotSimData(probes,caseInd,probeInd,lineDef,xVar,yVar,k_res = False)
    plotExpt(xExpt, yExpt, lineDef_Indices_Expt[lineDef]) 
    plotLabelEtc(xlabelText,ylabelText,plotSubDir,figName)

In [9]:
expDataFile = "measured_modified.txt"
simList = ['kEps_rans_asl_coarseMesh', \
           'kOmega_des_asl_coarseMesh', \
           'kOmega_iddes_asl_coarseMesh', \
           'kOmega_des_asl_fineMesh', \
           'kOmega_iddes_asl_fineMesh', \
           'kOmega_iddes_asl_betaStar0.03', \
           'kOmega_des_asl_betaStar0.03']

labelText = ['ke RANS', \
             'kw DES, beta*=0.09', \
             'kw IDDES, beta*=0.09', \
             'Fine, k-w DES', \
             'Fine, k-w IDDES',
             'kw DES, beta*=0.03', \
             'kw IDDES, beta*=0.03']
"""
labelText = ['k-e, RANS', \
             'Coarse,k-w DES', \
             'k-w, DES', \
             'Fine, k-w DES', \
             'Fine, k-w IDDES']
"""

#plotStyle = ['r-', 'b--', 'g-', 'm-.', 'k-', 'y--', 'r-.', 'k:']
plotStyle = ['r-*', 'b-->', 'g-.o', 'm-.^', 'k--*', 'y--^', 'k-.*', 'k:']

In [10]:
# Map plot subdirectories to case indices
plotSubDir_Indices = {'validation_all'  : [0, 1, 2, 5, 6], \
                      'validation_toPresent':[0, 2]}
# Map line definition to probe indices
lineDef_Indices_Expt = {'A' : [0, 1, 2, 3, 4, 6, 7, 8], \
                        'AASW': [9, 10, 11, 12, 13, 14, 15], \
                        'AANE': [17, 18, 19, 20, 21, 22], \
                        'B' : [24, 23, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38]}
lineDef_Indices_coarseMesh = {'A' : [0, 1, 2, 3, 4, 6, 7, 8], \
                              'AASW': [9, 10, 11, 12, 13, 14, 15], \
                              'AANE': [17, 18, 19, 20, 21], \
                              'B' : [23, 22, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35]}
lineDef_Indices_fineMesh = {'A' : [0, 1, 2, 3, 4, 6, 7, 8], \
                            'AASW': [9, 10, 11, 12, 13, 14], \
                            'AANE': [16, 17, 18, 19, 20, 21], \
                            'B' : [23, 22, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37]}

In [11]:
## Make sure to provide the correct reference values used in the experiment ##
U_ref = 8.0
TKE_ref = 75.0
sector = 210.0

expData = pd.read_csv(expDataFile, sep='\s+', skiprows=1,
                         names=['name1', 'name2', 'name3','x', 'y', 'H', 'dist', 'FSR', 'FSRmin', 'FSRmax', 'TKEstar'])
#print 'Expt data read: \n', expData

for i in range(len(expData['TKEstar'])):
    if expData['TKEstar'][i] < 0:
        expData['TKEstar'][i]  = 'nan'
#print 'TKE: ', expData['TKEstar']*TKE_ref
    
# Compute derived quantities from expt data
Uax = (1 + expData['FSR'])*U_ref
#print 'Uax: ', Uax

/home/pkjha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [12]:
# Sanity check to see if different lines were assigned correct indices 
for lineDef in lineDef_Indices_Expt.keys():
    probeInd = lineDef_Indices_Expt[lineDef]
    print ('... lineDef (Expt):', lineDef, '... probe index: ',probeInd, '...')
    #print '... Probe names:\n',expData.iloc[probeInd]['name1']

... lineDef (Expt): A ... probe index:  [0, 1, 2, 3, 4, 6, 7, 8] ...
... lineDef (Expt): AASW ... probe index:  [9, 10, 11, 12, 13, 14, 15] ...
... lineDef (Expt): AANE ... probe index:  [17, 18, 19, 20, 21, 22] ...
... lineDef (Expt): B ... probe index:  [24, 23, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38] ...


In [13]:
for lineDef in lineDef_Indices_Expt.keys():
    probeInd = lineDef_Indices_Expt[lineDef]
    plt.plot(expData.iloc[probeInd]['x'], expData.iloc[probeInd]['y'], '-*', label=lineDef)
plotLabelEtc('x (Met Coord)','y (Met Coord)','.','lineDef.png')

In [14]:
caseCount = 0
#test_path_arr = {}
probesU = {}
probesUp = {}
probesk = {}
probesUax = {}
probeInd = {}
caseName = {}
for sim in simList:
    #print 'Case count: ', caseCount
    case_name = 'ask_' + sim
    #print 'case_name: ', case_name
    caseName[caseCount] = case_name
    test_des = os.path.join(case_name, 'masts')

    #Determine the last step from DES results
    #test_timeDirs = map(int,os.listdir(test_des)) # Note the integer mapping
    test_timeDirs = os.listdir(test_des)
    test_timeDirs.sort()
    #print(test_timeDirs)
    test_lastTime = test_timeDirs[-1]
    #print 'Test results time dirs:\n', test_timeDirs, '\n Last time instant: ', test_lastTime, '\n'

    test_path = os.path.join(test_des, str(test_lastTime))
    #print "Test path: %s"%(test_path)
    #test_path_arr[caseCount] = test_path
    #caseCount += 1
    # Load DES results

    if sim == 'kEps_rans_asl_coarseMesh':
        probesU_test = readU(test_path, 'sensors_U.xy')
        probesk_test = readkEpsRANS(test_path, 'sensors_k_p.xy')
    else:
        probesU_test = readU(test_path, 'sensors_UMean.xy')
        probesk_test = readkOmega(test_path, 'sensors_kMean_pMean_omegaMean_nuSgsMean.xy')
        if os.path.exists(os.path.join(test_path, 'sensors_UPrime2Mean.xy')):
            probesUp_test = readUprime(test_path, 'sensors_UPrime2Mean.xy')
            probesUp[caseCount] = probesUp_test
            #print 'Test results, Up:\n',probesUp_test
    # Compute axial velocity from DES and results
    #Uax_des_test = axialVel(probesU_test['U'], probesU_test['V'], sector)

    #U[caseCount] = probesU_test['U'].tolist()
    probesU[caseCount] = probesU_test
    probesk[caseCount] = probesk_test
    probesUax[caseCount] = pd.DataFrame({'x':probesU_test['x'],\
                                         'Uax': axialVel(probesU_test['U'], probesU_test['V'], sector)})
    
    if 'coarseMesh' in sim or 'betaStar' in sim:
        probeInd[caseCount] = lineDef_Indices_coarseMesh
    else:
        probeInd[caseCount] = lineDef_Indices_fineMesh
    caseCount += 1

    #print 'Test results, U:\n',probesU_test
    
    #print 'Test results, k:\n',probesk_test
    #print  'Uax_DES_test :\n', Uax_des_test

In [15]:
#U = [probesU[i] for i in range(len(probesU))]
#print 'U: ', U
#print 'probesU: ', probesU
#print 'probesUp: ', probesUp
#print 'labelText', labelText
#print 'len(probesU): ', len(probesU)
#print 'len(labelText): ', len(labelText)
#print 'probe indices:', probeIndices

In [16]:
for plotSubDir in plotSubDir_Indices.keys():
    caseInd = plotSubDir_Indices[plotSubDir]
    print ('plotSubDir:', plotSubDir)
    for lineDef in lineDef_Indices_Expt.keys():
        print ('... lineDef:', lineDef)
        #plotVar(probesU,caseInd,probeInd,lineDef,'x','U','x(Met Coord)','Mean Ux (m/s)',plotSubDir,'line_'+lineDef+'_Ux.png')
        #plotVar(probesU,caseInd,probeInd,lineDef,'x','V','x(Met Coord)','Mean Uy (m/s)',plotSubDir,'line_'+lineDef+'_Uy.png')
        #plotVar(probesU,caseInd,probeInd,lineDef,'x','W','x(Met Coord)','Mean Uz (m/s)',plotSubDir,'line_'+lineDef+'_Uz.png')
        #plotVar(probesk,caseInd,probeInd,lineDef,'x','p','x(Met Coord)','pMean (N/m^2)',plotSubDir,'line_'+lineDef+'_p.png')
        plotVarExpt(probesUax,caseInd,probeInd,lineDef,'x', 'Uax', 'x(Met Coord)', 'Mean Uaxial (m/s)',plotSubDir,'line_'+lineDef+'_Uax.png', expData['x'], Uax)
        if lineDef == 'B':
            plotVar(probesk,caseInd,probeInd,lineDef,'x','k', 'x(Met Coord)', 'Mean TKE (m^2/s^2)',plotSubDir,'line_'+lineDef+'_k.png', True)
        else:
            plotVarExpt(probesk,caseInd,probeInd,lineDef,'x','k', 'x(Met Coord)', 'Mean TKE (m^2/s^2)',plotSubDir,'line_'+lineDef+'_k.png', expData['x'], expData['TKEstar']*TKE_ref, True)

plotSubDir: validation_all
... lineDef: A
... lineDef: AASW
... lineDef: AANE
... lineDef: B
plotSubDir: validation_toPresent
... lineDef: A
... lineDef: AASW
... lineDef: AANE
... lineDef: B
